## Which Cities are Similar to Rome To Start New Business?

In [15]:
import pandas as pd
import numpy as np
import requests


In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

#### Reading data of Cities of Interest and generating address

In [3]:
countries=pd.read_csv('Countries.csv',delimiter=';')
countries.head()

,Cities,Countries
0,Mumbai,India
1,Delhi,India
2,Madrid,Spain
3,Los Angeles,USA
4,Las Vegas,USA


In [4]:
countries['Address']=countries['Cities'] +", " + countries['Countries']

In [5]:
countries.head()

,Cities,Countries,Address
0,Mumbai,India,"Mumbai, India"
1,Delhi,India,"Delhi, India"
2,Madrid,Spain,"Madrid, Spain"
3,Los Angeles,USA,"Los Angeles, USA"
4,Las Vegas,USA,"Las Vegas, USA"


#### Generating Lat/Long Details of the Cities

In [6]:
from geopy.geocoders import Nominatim

In [7]:
to_drop_unknown = []
geolocator = Nominatim(user_agent="coursera")
for index, row in countries.iterrows():
    address = row['Address']
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        countries.loc[index, 'Latitude'] = latitude
        countries.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

countries.head()

The geograpical coordinate of Mumbai, India are 18.9387711, 72.8353355.
The geograpical coordinate of Delhi, India are 28.6517178, 77.2219388.
The geograpical coordinate of Madrid, Spain are 40.4167047, -3.7035825.
The geograpical coordinate of Los Angeles, USA are 34.0536909, -118.2427666.
The geograpical coordinate of Las Vegas, USA are 36.1672559, -115.1485163.
The geograpical coordinate of Tokyo, Japan are 35.6828387, 139.7594549.
The geograpical coordinate of Hongkong, China are 22.2793278, 114.1628131.
The geograpical coordinate of Paris, France are 48.8566969, 2.3514616.
The geograpical coordinate of Rome, Italy are 41.8933203, 12.4829321.
The geograpical coordinate of Dublin, South Africa are -26.182254, 27.9814051.
The geograpical coordinate of Dhaka, Bangladesh are 23.7593572, 90.3788136.
The geograpical coordinate of Perth, Australia are -31.9527121, 115.8604796.
The geograpical coordinate of Moscow, Russia are 55.4792046, 37.3273304.
The geograpical coordinate of Sydney, Au

,Cities,Countries,Address,Latitude,Longitude
0,Mumbai,India,"Mumbai, India",18.938771,72.835335
1,Delhi,India,"Delhi, India",28.651718,77.221939
2,Madrid,Spain,"Madrid, Spain",40.416705,-3.703582
3,Los Angeles,USA,"Los Angeles, USA",34.053691,-118.242767
4,Las Vegas,USA,"Las Vegas, USA",36.167256,-115.148516


#### Getting Nearby Venues for each City. We drop the cities for which data not available

In [12]:
countries_final=countries.drop(to_drop_unknown)
CLIENT_ID = 'K0HG2QN0P4TMKQALWXT2QQJNDLV1BZBWVAQVVLO3IK4ANHSO'
CLIENT_SECRET = 'GAOL4UL0YO3VRD0YTKI4CYYBRGDD42YHIDPGS4VGTJWZXLP5' 
VERSION = '20180605'
LIMIT=100

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
global_venues = getNearbyVenues(names=countries_final['Cities'],
                                   latitudes=countries_final['Latitude'],
                                   longitudes=countries_final['Longitude']
                                  )

Mumbai
Delhi
Madrid
Los Angeles
Las Vegas
Tokyo
Hongkong
Paris
Rome
Dublin
Dhaka
Perth
Moscow
Sydney
Melbourne
Washington DC
Seol
Istanbul
Mexico City
Cairo
Buenos Aires
Manila
London


In [17]:
global_venues.head()

,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mumbai,18.938771,72.835335,Town House Cafe,18.938550,72.833464,Bar
1,Mumbai,18.938771,72.835335,Royal China,18.938715,72.832933,Chinese Restaurant
2,Mumbai,18.938771,72.835335,Sher-E-Punjab,18.937944,72.837853,Indian Restaurant
3,Mumbai,18.938771,72.835335,Cafe Excelsior,18.937701,72.833566,Café
4,Mumbai,18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train Station


#### One Hot Encoding and City wise Grouping for K Means

In [18]:
global_onehot = pd.get_dummies(global_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
global_onehot['City'] = global_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [global_onehot.columns[-1]] + list(global_onehot.columns[:-1])
global_onehot = global_onehot[fixed_columns]

global_onehot.head()

,City,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wedding Hall,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Mumbai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mumbai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Mumbai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mumbai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mumbai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
global_grouped = global_onehot.groupby('City').mean().reset_index()
global_grouped

,City,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wedding Hall,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Buenos Aires,0.000000,0.00,0.000000,0.04,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Cairo,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.020833,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Dhaka,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Dublin,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Hongkong,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.018519,0.000000,0.000000,...,0.000000,0.018519,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.018519,0.018519
5,Istanbul,0.000000,0.00,0.000000,0.00,0.014085,0.000000,0.000000,0.000000,0.014085,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Las Vegas,0.000000,0.00,0.038462,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.038462,0.00,0.038462,0.000000,0.000000,0.000000,0.000000
7,London,0.012821,0.00,0.000000,0.00,0.000000,0.000000,0.038462,0.012821,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.025641,0.000000,0.000000
8,Los Angeles,0.000000,0.00,0.037037,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Madrid,0.000000,0.00,0.000000,0.00,0.000000,0.022727,0.011364,0.034091,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.034091,0.000000,0.000000


In [20]:
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans


import folium 
print('Libraries imported.')

Libraries imported.


#### K means analysis

In [21]:

kclusters = 6

global_grouped_clustering = global_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(global_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:25]

array([4, 4, 1, 3, 4, 4, 2, 4, 4, 4, 4, 4, 4, 5, 0, 4, 0, 4, 2, 0, 4],
      dtype=int32)

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = global_grouped['City']

for ind in np.arange(global_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(global_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buenos Aires,Argentinian Restaurant,Pizza Place,Bakery,Ice Cream Shop,Park,Coffee Shop,Science Museum,Café,Cafeteria,Gym
1,Cairo,Café,Egyptian Restaurant,Coffee Shop,Pastry Shop,Fast Food Restaurant,Theater,Italian Restaurant,Pie Shop,Sports Club,Bookstore
2,Dhaka,Café,Plaza,Scenic Lookout,Food,Arts & Crafts Store,Falafel Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit
3,Dublin,Bakery,Spa,Art Gallery,Clothing Store,Flower Shop,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
4,Hongkong,Hotel,Café,Lounge,Steakhouse,Cantonese Restaurant,Park,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant,Bookstore


In [24]:
countries_final

,Cities,Countries,Address,Latitude,Longitude
0,Mumbai,India,"Mumbai, India",18.938771,72.835335
1,Delhi,India,"Delhi, India",28.651718,77.221939
2,Madrid,Spain,"Madrid, Spain",40.416705,-3.703582
3,Los Angeles,USA,"Los Angeles, USA",34.053691,-118.242767
4,Las Vegas,USA,"Las Vegas, USA",36.167256,-115.148516
5,Tokyo,Japan,"Tokyo, Japan",35.682839,139.759455
6,Hongkong,China,"Hongkong, China",22.279328,114.162813
7,Paris,France,"Paris, France",48.856697,2.351462
8,Rome,Italy,"Rome, Italy",41.893320,12.482932
9,Dublin,South Africa,"Dublin, South Africa",-26.182254,27.981405


In [25]:
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = countries_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_venues_sorted.set_index('City'), on='Cities')

city_merged.head() 

,Cities,Countries,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mumbai,India,"Mumbai, India",18.938771,72.835335,5.0,Indian Restaurant,Bar,Coffee Shop,Irani Cafe,Lounge,Chinese Restaurant,Café,Seafood Restaurant,Train Station,Athletics & Sports
1,Delhi,India,"Delhi, India",28.651718,77.221939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Madrid,Spain,"Madrid, Spain",40.416705,-3.703582,4.0,Hotel,Hostel,Plaza,Restaurant,Spanish Restaurant,Wine Bar,Gourmet Shop,Art Museum,Tapas Restaurant,Peruvian Restaurant
3,Los Angeles,USA,"Los Angeles, USA",34.053691,-118.242767,4.0,Museum,Shopping Mall,Pop-Up Shop,Sushi Restaurant,Sandwich Place,Candy Store,Bookstore,Noodle House,Men's Store,Speakeasy
4,Las Vegas,USA,"Las Vegas, USA",36.167256,-115.148516,2.0,Theater,Coffee Shop,Hotel,Casino,Italian Restaurant,Concert Hall,Pizza Place,Park,Sandwich Place,Bus Station


In [26]:
city_merged

,Cities,Countries,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mumbai,India,"Mumbai, India",18.938771,72.835335,5.0,Indian Restaurant,Bar,Coffee Shop,Irani Cafe,Lounge,Chinese Restaurant,Café,Seafood Restaurant,Train Station,Athletics & Sports
1,Delhi,India,"Delhi, India",28.651718,77.221939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Madrid,Spain,"Madrid, Spain",40.416705,-3.703582,4.0,Hotel,Hostel,Plaza,Restaurant,Spanish Restaurant,Wine Bar,Gourmet Shop,Art Museum,Tapas Restaurant,Peruvian Restaurant
3,Los Angeles,USA,"Los Angeles, USA",34.053691,-118.242767,4.0,Museum,Shopping Mall,Pop-Up Shop,Sushi Restaurant,Sandwich Place,Candy Store,Bookstore,Noodle House,Men's Store,Speakeasy
4,Las Vegas,USA,"Las Vegas, USA",36.167256,-115.148516,2.0,Theater,Coffee Shop,Hotel,Casino,Italian Restaurant,Concert Hall,Pizza Place,Park,Sandwich Place,Bus Station
5,Tokyo,Japan,"Tokyo, Japan",35.682839,139.759455,0.0,Historic Site,Convenience Store,Café,Italian Restaurant,Japanese Restaurant,French Restaurant,Park,Chinese Restaurant,Bar,Hotel Bar
6,Hongkong,China,"Hongkong, China",22.279328,114.162813,4.0,Hotel,Café,Lounge,Steakhouse,Cantonese Restaurant,Park,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant,Bookstore
7,Paris,France,"Paris, France",48.856697,2.351462,0.0,French Restaurant,Ice Cream Shop,Plaza,Creperie,Historic Site,Art Gallery,Italian Restaurant,Hotel,Tea Room,Clothing Store
8,Rome,Italy,"Rome, Italy",41.893320,12.482932,0.0,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,History Museum,Plaza,Ice Cream Shop
9,Dublin,South Africa,"Dublin, South Africa",-26.182254,27.981405,3.0,Bakery,Spa,Art Gallery,Clothing Store,Flower Shop,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


In [27]:
city_merged. dropna(subset = ["Cluster Labels"], inplace=True)

In [28]:
city_merged['Cluster Labels'] = city_merged['Cluster Labels'].apply(np.int64)
city_merged

,Cities,Countries,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mumbai,India,"Mumbai, India",18.938771,72.835335,5,Indian Restaurant,Bar,Coffee Shop,Irani Cafe,Lounge,Chinese Restaurant,Café,Seafood Restaurant,Train Station,Athletics & Sports
2,Madrid,Spain,"Madrid, Spain",40.416705,-3.703582,4,Hotel,Hostel,Plaza,Restaurant,Spanish Restaurant,Wine Bar,Gourmet Shop,Art Museum,Tapas Restaurant,Peruvian Restaurant
3,Los Angeles,USA,"Los Angeles, USA",34.053691,-118.242767,4,Museum,Shopping Mall,Pop-Up Shop,Sushi Restaurant,Sandwich Place,Candy Store,Bookstore,Noodle House,Men's Store,Speakeasy
4,Las Vegas,USA,"Las Vegas, USA",36.167256,-115.148516,2,Theater,Coffee Shop,Hotel,Casino,Italian Restaurant,Concert Hall,Pizza Place,Park,Sandwich Place,Bus Station
5,Tokyo,Japan,"Tokyo, Japan",35.682839,139.759455,0,Historic Site,Convenience Store,Café,Italian Restaurant,Japanese Restaurant,French Restaurant,Park,Chinese Restaurant,Bar,Hotel Bar
6,Hongkong,China,"Hongkong, China",22.279328,114.162813,4,Hotel,Café,Lounge,Steakhouse,Cantonese Restaurant,Park,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant,Bookstore
7,Paris,France,"Paris, France",48.856697,2.351462,0,French Restaurant,Ice Cream Shop,Plaza,Creperie,Historic Site,Art Gallery,Italian Restaurant,Hotel,Tea Room,Clothing Store
8,Rome,Italy,"Rome, Italy",41.893320,12.482932,0,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,History Museum,Plaza,Ice Cream Shop
9,Dublin,South Africa,"Dublin, South Africa",-26.182254,27.981405,3,Bakery,Spa,Art Gallery,Clothing Store,Flower Shop,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
10,Dhaka,Bangladesh,"Dhaka, Bangladesh",23.759357,90.378814,1,Café,Plaza,Scenic Lookout,Food,Arts & Crafts Store,Falafel Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit


##### Visualizing the Clusters

In [29]:
map_clusters = folium.Map(zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'],city_merged['Longitude'], city_merged['Cities'], 
                                  city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Tokyo,0,Historic Site,Convenience Store,Café,Italian Restaurant,Japanese Restaurant,French Restaurant,Park,Chinese Restaurant,Bar,Hotel Bar
7,Paris,0,French Restaurant,Ice Cream Shop,Plaza,Creperie,Historic Site,Art Gallery,Italian Restaurant,Hotel,Tea Room,Clothing Store
8,Rome,0,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,History Museum,Plaza,Ice Cream Shop


In [61]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Dhaka,1,Café,Plaza,Scenic Lookout,Food,Arts & Crafts Store,Falafel Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit


In [62]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Las Vegas,2,Theater,Coffee Shop,Hotel,Casino,Italian Restaurant,Concert Hall,Pizza Place,Park,Sandwich Place,Bus Station
13,Sydney,2,Theater,Concert Hall,Australian Restaurant,Cocktail Bar,Boat or Ferry,Coffee Shop,Café,Pier,Park,Opera House


In [63]:
city_merged.loc[city_merged['Cluster Labels'] == 3, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Dublin,3,Bakery,Spa,Art Gallery,Clothing Store,Flower Shop,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


In [64]:
city_merged.loc[city_merged['Cluster Labels'] == 4, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Madrid,4,Hotel,Hostel,Plaza,Restaurant,Spanish Restaurant,Wine Bar,Gourmet Shop,Art Museum,Tapas Restaurant,Peruvian Restaurant
3,Los Angeles,4,Museum,Shopping Mall,Pop-Up Shop,Sushi Restaurant,Sandwich Place,Candy Store,Bookstore,Noodle House,Men's Store,Speakeasy
6,Hongkong,4,Hotel,Café,Lounge,Steakhouse,Cantonese Restaurant,Park,Dim Sum Restaurant,Chinese Restaurant,Italian Restaurant,Bookstore
11,Perth,4,Coffee Shop,Hotel,Café,Korean Restaurant,Wine Bar,Sushi Restaurant,Vietnamese Restaurant,Restaurant,Bar,Indian Restaurant
14,Melbourne,4,Coffee Shop,Café,Bar,Cocktail Bar,Dessert Shop,Shopping Mall,Toy / Game Store,Burger Joint,Candy Store,Chocolate Shop
15,Washington DC,4,Garden,Hotel,Outdoor Sculpture,Bar,Government Building,Sandwich Place,Monument / Landmark,American Restaurant,Hotel Bar,Food Truck
16,Seol,4,Coffee Shop,Korean Restaurant,Chinese Restaurant,Hotel,Historic Site,Lounge,Café,Japanese Restaurant,Bookstore,Plaza
17,Istanbul,4,Hotel,Turkish Restaurant,Café,Coffee Shop,Kebab Restaurant,Historic Site,Restaurant,Pilavcı,Mosque,Gym
18,Mexico City,4,Art Museum,Museum,Arts & Crafts Store,Ice Cream Shop,Boutique,Hotel,Historic Site,Department Store,Mexican Restaurant,Restaurant
19,Cairo,4,Café,Egyptian Restaurant,Coffee Shop,Pastry Shop,Fast Food Restaurant,Theater,Italian Restaurant,Pie Shop,Sports Club,Bookstore


In [65]:
city_merged.loc[city_merged['Cluster Labels'] == 5, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]

,Cities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mumbai,5,Indian Restaurant,Bar,Coffee Shop,Irani Cafe,Lounge,Chinese Restaurant,Café,Seafood Restaurant,Train Station,Athletics & Sports
